In [1]:
import pandas as pd
from data_disaggregation import apply_map_df, VarTypeCategorical, VarTypeMetric, VarTypeMetricExt, VarTypeOrdinal

In [2]:
ds = pd.Series([10]).rename_axis(['year'])
ds_map = pd.Series({(0, 0): 1, (0, 1): 3}).rename_axis(['year',"month"])

apply_map_df(vtype=VarTypeMetricExt, s_var=ds,s_map=ds_map)


month
0    2.5
1    7.5
dtype: float64

In [3]:
VarTypeMetricExt.apply_map_df(s_var=ds,s_map=ds_map)

month
0    2.5
1    7.5
dtype: float64